# 📊 Análise Exploratória dos Dados WIN$N_M1 

## 🎯 Objetivo
Este notebook realiza uma **análise exploratória completa** dos dados brutos do arquivo `WIN$N_M1.csv`, que contém informações financeiras do **Mini Índice Bovespa (WIN$N)** em timeframe de **1 minuto**.

## 📋 Estrutura da Análise
1. **Carregamento e Exploração Inicial** dos dados
2. **Descrição Detalhada das Variáveis** financeiras
3. **Estatísticas Descritivas** e análise de qualidade
4. **Visualizações Temporais** dos preços e volumes
5. **Análise de Distribuições** e outliers
6. **Correlações** entre variáveis
7. **Insights** e padrões identificados

---

## 🔍 Sobre o WIN$N
O **WIN$N** é o **Mini Índice Bovespa**, um contrato futuro que replica o Ibovespa com menor valor de tick. É amplamente utilizado para:
- **Day Trading** e operações de curto prazo
- **Hedge** de carteiras de ações
- **Arbitragem** entre mercado à vista e futuro

In [ ]:
# 📦 Importação das Bibliotecas Necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configurações para visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configurações do matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

print("✅ Bibliotecas importadas com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")

In [ ]:
# 📂 Carregamento dos Dados WIN$N_M1.csv
try:
    # Tentativa de carregamento do arquivo
    dados_brutos = pd.read_csv('../WIN$N_M1.csv')
    print("✅ Arquivo WIN$N_M1.csv carregado com sucesso!")
    
    # Informações básicas do dataset
    print("\n" + "="*50)
    print("📊 INFORMAÇÕES BÁSICAS DO DATASET")
    print("="*50)
    print(f"📏 Dimensões: {dados_brutos.shape}")
    print(f"📅 Colunas: {dados_brutos.shape[1]}")
    print(f"📈 Registros: {dados_brutos.shape[0]:,}")
    
    # Primeiras linhas do dataset
    print(f"\n🔍 Primeiras 5 linhas:")
    print(dados_brutos.head())
    
except FileNotFoundError:
    print("❌ Erro: Arquivo WIN$N_M1.csv não encontrado!")
    print("📁 Verifique se o arquivo está na pasta correta.")
except Exception as e:
    print(f"❌ Erro ao carregar arquivo: {e}")

## 📋 Descrição Detalhada das Variáveis do Dataset

O arquivo **WIN$N_M1.csv** contém dados financeiros do Mini Índice Bovespa em timeframe de **1 minuto**. Cada linha representa um minuto de negociação com as seguintes informações:

### 🔢 **Variáveis Presentes:**

| Variável | Descrição | Significado Financeiro |
|----------|-----------|------------------------|
| **📅 Data/Hora** | Timestamp do minuto | Momento exato da negociação (YYYY-MM-DD HH:MM:SS) |
| **💰 Open (Abertura)** | Preço de abertura do minuto | Primeiro preço negociado no período |
| **📊 High (Máxima)** | Preço máximo do minuto | Maior preço atingido no período |
| **📉 Low (Mínima)** | Preço mínimo do minuto | Menor preço atingido no período |
| **🎯 Close (Fechamento)** | Preço de fechamento do minuto | Último preço negociado no período |
| **📦 Volume** | Volume negociado | Quantidade de contratos negociados |

### 🏦 **Características do WIN$N:**
- **Tick Mínimo**: R$ 5,00 por ponto
- **Horário**: 09:00 às 18:00 (com after market até 18:30)
- **Liquidação**: Financeira pela média do Ibovespa
- **Margem**: Definida pela B3 (bolsa brasileira)

In [ ]:
# 🔍 Exploração Detalhada da Estrutura dos Dados
print("📊 INFORMAÇÕES DETALHADAS DO DATASET")
print("="*50)

# Informações sobre tipos de dados
print(f"\n📋 Informações dos tipos de dados:")
print(dados_brutos.info())

print(f"\n📏 Estatísticas de formato:")
print(f"   • Memória utilizada: {dados_brutos.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Verificar colunas disponíveis
print(f"\n📂 Colunas disponíveis:")
for i, col in enumerate(dados_brutos.columns, 1):
    print(f"   {i}. {col}")

# Verificar valores nulos
print(f"\n🔍 Verificação de valores nulos:")
valores_nulos = dados_brutos.isnull().sum()
for col in dados_brutos.columns:
    nulos = valores_nulos[col]
    porcentagem = (nulos / len(dados_brutos)) * 100
    print(f"   • {col}: {nulos} nulos ({porcentagem:.2f}%)")

# Verificar duplicatas
duplicatas = dados_brutos.duplicated().sum()
print(f"\n🔄 Registros duplicados: {duplicatas}")

print(f"\n✅ Análise da estrutura concluída!")

In [ ]:
# 📊 Estatísticas Descritivas Completas
print("📈 ESTATÍSTICAS DESCRITIVAS - DADOS BRUTOS WIN$N")
print("="*60)

# Identificar colunas numéricas (preços e volume)
colunas_numericas = dados_brutos.select_dtypes(include=[np.number]).columns

if len(colunas_numericas) > 0:
    # Estatísticas descritivas básicas
    stats_desc = dados_brutos[colunas_numericas].describe()
    print("\n📋 Estatísticas Básicas:")
    print(stats_desc.round(2))
    
    # Estatísticas adicionais
    print(f"\n📊 Métricas Adicionais:")
    for col in colunas_numericas:
        dados_col = dados_brutos[col]
        print(f"\n   📊 {col}:")
        print(f"      • Mediana: {dados_col.median():.2f}")
        print(f"      • Moda: {dados_col.mode().iloc[0] if len(dados_col.mode()) > 0 else 'N/A'}")
        print(f"      • Desvio Padrão: {dados_col.std():.2f}")
        print(f"      • Coeficiente de Variação: {(dados_col.std()/dados_col.mean())*100:.2f}%")
        print(f"      • Assimetria (Skewness): {dados_col.skew():.3f}")
        print(f"      • Curtose (Kurtosis): {dados_col.kurtosis():.3f}")
        
        # Amplitude
        amplitude = dados_col.max() - dados_col.min()
        print(f"      • Amplitude: {amplitude:.2f}")
        
        # Quartis
        q1 = dados_col.quantile(0.25)
        q3 = dados_col.quantile(0.75)
        iqr = q3 - q1
        print(f"      • IQR (Amplitude Interquartil): {iqr:.2f}")

else:
    print("❌ Nenhuma coluna numérica identificada automaticamente.")
    print("🔧 Verificando estrutura manual...")
    print(dados_brutos.dtypes)

In [ ]:
# 📈 Preparação dos Dados para Visualização
# Tentativa de identificar e converter colunas de preços

# Primeiro, vamos examinar as primeiras linhas para entender a estrutura
print("🔍 Examinando estrutura dos dados:")
print("\nPrimeiras 3 linhas:")
print(dados_brutos.head(3))

print("\nTipos de dados:")
print(dados_brutos.dtypes)

# Tentar identificar colunas que podem ser numéricas
# Assumindo padrão comum de dados financeiros: Date, Open, High, Low, Close, Volume
possivel_estrutura = {
    'data_hora': 0,  # Primeira coluna geralmente é timestamp
    'open': 1,       # Abertura
    'high': 2,       # Máxima  
    'low': 3,        # Mínima
    'close': 4,      # Fechamento
    'volume': 5      # Volume
}

# Criar uma cópia dos dados para trabalhar
dados_processados = dados_brutos.copy()

# Tentar converter colunas numéricas se necessário
colunas = dados_processados.columns.tolist()
print(f"\n📋 Colunas disponíveis: {colunas}")

# Se tivermos pelo menos 5 colunas, assumir estrutura OHLCV
if len(colunas) >= 5:
    try:
        # Tentar converter colunas 1-5 para numéricas (assumindo que 0 é timestamp)
        for i in range(1, min(6, len(colunas))):
            dados_processados.iloc[:, i] = pd.to_numeric(dados_processados.iloc[:, i], errors='coerce')
        
        print("✅ Conversão numérica realizada!")
        
        # Renomear colunas para facilitar análise
        if len(colunas) >= 6:
            dados_processados.columns = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume'][:len(colunas)]
        
        print(f"📊 Estrutura após processamento:")
        print(dados_processados.dtypes)
        
    except Exception as e:
        print(f"❌ Erro na conversão: {e}")
        print("🔧 Mantendo dados originais para análise manual")

# Verificar se temos dados numéricos agora
colunas_numericas_final = dados_processados.select_dtypes(include=[np.number]).columns
print(f"\n✅ Colunas numéricas identificadas: {list(colunas_numericas_final)}")

In [ ]:
# 📊 Visualização 1: Série Temporal dos Preços
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📈 Análise Temporal dos Preços WIN$N - Dados Brutos', fontsize=16, fontweight='bold')

# Usar amostra dos dados para visualização (primeiros 1000 registros)
amostra_size = min(1000, len(dados_processados))
amostra = dados_processados.head(amostra_size)

# Identificar colunas de preços (assumindo estrutura OHLC)
if len(dados_processados.columns) >= 5:
    try:
        # Subplot 1: Preços OHLC
        axes[0,0].plot(amostra.index, amostra.iloc[:, 1], label='Open', alpha=0.7, color='blue')
        axes[0,0].plot(amostra.index, amostra.iloc[:, 2], label='High', alpha=0.7, color='green')
        axes[0,0].plot(amostra.index, amostra.iloc[:, 3], label='Low', alpha=0.7, color='red')
        axes[0,0].plot(amostra.index, amostra.iloc[:, 4], label='Close', alpha=0.8, color='black', linewidth=2)
        axes[0,0].set_title('💰 Preços OHLC (Primeiros 1000 registros)')
        axes[0,0].set_xlabel('Índice do Registro')
        axes[0,0].set_ylabel('Preço (pontos)')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        
        # Subplot 2: Volume
        if len(dados_processados.columns) >= 6:
            axes[0,1].bar(amostra.index, amostra.iloc[:, 5], alpha=0.6, color='orange')
            axes[0,1].set_title('📦 Volume Negociado')
            axes[0,1].set_xlabel('Índice do Registro')
            axes[0,1].set_ylabel('Volume')
            axes[0,1].grid(True, alpha=0.3)
        
        # Subplot 3: Spread (High - Low)
        spread = amostra.iloc[:, 2] - amostra.iloc[:, 3]
        axes[1,0].plot(amostra.index, spread, color='purple', alpha=0.7)
        axes[1,0].set_title('📏 Spread (High - Low)')
        axes[1,0].set_xlabel('Índice do Registro')
        axes[1,0].set_ylabel('Spread (pontos)')
        axes[1,0].grid(True, alpha=0.3)
        
        # Subplot 4: Retornos (Close to Close)
        if len(amostra) > 1:
            retornos = amostra.iloc[:, 4].pct_change().dropna()
            axes[1,1].plot(retornos.index, retornos * 100, color='darkorange', alpha=0.7)
            axes[1,1].set_title('📈 Retornos Percentuais (Close-to-Close)')
            axes[1,1].set_xlabel('Índice do Registro')
            axes[1,1].set_ylabel('Retorno (%)')
            axes[1,1].grid(True, alpha=0.3)
            axes[1,1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
        
    except Exception as e:
        print(f"❌ Erro na visualização: {e}")
        # Fallback: mostrar dados como estão
        for i, ax in enumerate(axes.flat):
            if i < len(dados_processados.columns):
                ax.plot(amostra.index, amostra.iloc[:, i])
                ax.set_title(f'Coluna {i}: {dados_processados.columns[i]}')

plt.tight_layout()
plt.show()

print(f"📊 Gráfico gerado com amostra de {amostra_size} registros dos dados brutos")

In [ ]:
# 📊 Visualização 2: Análise de Distribuições
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('📊 Análise de Distribuições - Dados WIN$N Brutos', fontsize=16, fontweight='bold')

# Verificar se temos dados numéricos para plotar
colunas_numericas = dados_processados.select_dtypes(include=[np.number]).columns

if len(colunas_numericas) >= 4:
    try:
        # Assumir estrutura OHLCV (Open, High, Low, Close, Volume)
        precos = ['Open', 'High', 'Low', 'Close'] if 'Open' in colunas_numericas else colunas_numericas[:4]
        
        # Histogramas dos preços
        for i, preco in enumerate(precos):
            if i < 4 and preco in dados_processados.columns:
                # Histograma
                axes[0, i].hist(dados_processados[preco], bins=50, alpha=0.7, edgecolor='black')
                axes[0, i].set_title(f'📊 Distribuição {preco}')
                axes[0, i].set_xlabel('Preço (pontos)')
                axes[0, i].set_ylabel('Frequência')
                axes[0, i].grid(True, alpha=0.3)
                
                # Adicionar estatísticas no gráfico
                media = dados_processados[preco].mean()
                mediana = dados_processados[preco].median()
                axes[0, i].axvline(media, color='red', linestyle='--', label=f'Média: {media:.2f}')
                axes[0, i].axvline(mediana, color='green', linestyle='--', label=f'Mediana: {mediana:.2f}')
                axes[0, i].legend()
        
        # Boxplots
        boxplot_data = [dados_processados[col].dropna() for col in precos if col in dados_processados.columns]
        boxplot_labels = [col for col in precos if col in dados_processados.columns]
        
        if boxplot_data:
            axes[1, 0].boxplot(boxplot_data, labels=boxplot_labels)
            axes[1, 0].set_title('📦 Boxplot - Preços OHLC')
            axes[1, 0].set_ylabel('Preço (pontos)')
            axes[1, 0].grid(True, alpha=0.3)
            axes[1, 0].tick_params(axis='x', rotation=45)
        
        # Volume (se disponível)
        if 'Volume' in dados_processados.columns:
            axes[1, 1].hist(dados_processados['Volume'], bins=50, alpha=0.7, color='orange', edgecolor='black')
            axes[1, 1].set_title('📦 Distribuição do Volume')
            axes[1, 1].set_xlabel('Volume')
            axes[1, 1].set_ylabel('Frequência')
            axes[1, 1].grid(True, alpha=0.3)
        
        # QQ-plot para normalidade (Close)
        if 'Close' in dados_processados.columns:
            from scipy import stats
            stats.probplot(dados_processados['Close'].dropna(), dist="norm", plot=axes[1, 2])
            axes[1, 2].set_title('📈 Q-Q Plot - Preço Close')
            axes[1, 2].grid(True, alpha=0.3)
            
    except Exception as e:
        print(f"❌ Erro na análise de distribuições: {e}")
        # Fallback simples
        for i, col in enumerate(colunas_numericas[:6]):
            row, col_idx = divmod(i, 3)
            if row < 2:
                axes[row, col_idx].hist(dados_processados[col], bins=30, alpha=0.7)
                axes[row, col_idx].set_title(f'Distribuição {col}')

else:
    print("❌ Dados numéricos insuficientes para análise de distribuições")
    # Mostrar informações sobre os dados
    for i, ax in enumerate(axes.flat):
        ax.text(0.5, 0.5, f'Coluna {i}\nVerificar tipos de dados', 
                ha='center', va='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()

print("📊 Análise de distribuições concluída!")

In [ ]:
# 🔗 Análise de Correlações entre Variáveis
print("🔗 ANÁLISE DE CORRELAÇÕES")
print("="*40)

# Calcular matriz de correlação para variáveis numéricas
colunas_numericas = dados_processados.select_dtypes(include=[np.number]).columns

if len(colunas_numericas) >= 2:
    # Matriz de correlação
    matriz_correlacao = dados_processados[colunas_numericas].corr()
    
    # Visualização da matriz de correlação
    plt.figure(figsize=(12, 8))
    
    # Heatmap
    mask = np.triu(np.ones_like(matriz_correlacao, dtype=bool))
    sns.heatmap(matriz_correlacao, 
                mask=mask,
                annot=True, 
                cmap='RdYlBu_r', 
                center=0,
                square=True,
                vmin=-1, vmax=1,
                cbar_kws={'label': 'Correlação'})
    
    plt.title('🔗 Matriz de Correlação - Variáveis WIN$N', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Análise das correlações mais fortes
    print("\n📊 Correlações mais relevantes:")
    
    # Encontrar correlações mais fortes (excluindo diagonal)
    correlacoes_flat = []
    for i in range(len(matriz_correlacao.columns)):
        for j in range(i+1, len(matriz_correlacao.columns)):
            var1 = matriz_correlacao.columns[i]
            var2 = matriz_correlacao.columns[j]
            corr_value = matriz_correlacao.iloc[i, j]
            correlacoes_flat.append((var1, var2, corr_value))
    
    # Ordenar por valor absoluto da correlação
    correlacoes_flat.sort(key=lambda x: abs(x[2]), reverse=True)
    
    print("\n🔝 Top 10 correlações:")
    for i, (var1, var2, corr) in enumerate(correlacoes_flat[:10], 1):
        print(f"   {i:2d}. {var1} ↔ {var2}: {corr:.4f}")
    
    # Interpretação das correlações
    print(f"\n📋 Interpretação:")
    print(f"   • Correlações > 0.8: Correlação muito forte")
    print(f"   • Correlações 0.6-0.8: Correlação forte") 
    print(f"   • Correlações 0.3-0.6: Correlação moderada")
    print(f"   • Correlações < 0.3: Correlação fraca")
    
else:
    print("❌ Dados numéricos insuficientes para análise de correlação")
    print(f"Colunas numéricas encontradas: {list(colunas_numericas)}")

print("\n✅ Análise de correlações concluída!")

In [ ]:
# 📊 Análise Avançada: Detecção de Outliers e Padrões
print("🔍 ANÁLISE AVANÇADA DE OUTLIERS E PADRÕES")
print("="*50)

colunas_numericas = dados_processados.select_dtypes(include=[np.number]).columns

if len(colunas_numericas) >= 1:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🔍 Análise de Outliers e Padrões Avançados', fontsize=16, fontweight='bold')
    
    # Focar na coluna de fechamento (Close) se disponível
    coluna_principal = 'Close' if 'Close' in dados_processados.columns else colunas_numericas[0]
    dados_principais = dados_processados[coluna_principal].dropna()
    
    # 1. Detecção de outliers usando IQR
    Q1 = dados_principais.quantile(0.25)
    Q3 = dados_principais.quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = dados_principais[(dados_principais < limite_inferior) | (dados_principais > limite_superior)]
    
    # Gráfico 1: Boxplot com outliers destacados
    axes[0, 0].boxplot(dados_principais, vert=True)
    axes[0, 0].set_title(f'📦 Boxplot {coluna_principal} com Outliers')
    axes[0, 0].set_ylabel('Valor')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Gráfico 2: Série temporal com outliers destacados
    axes[0, 1].plot(dados_principais.index, dados_principais, alpha=0.7, color='blue', label='Dados normais')
    if len(outliers) > 0:
        axes[0, 1].scatter(outliers.index, outliers, color='red', s=50, alpha=0.8, label=f'Outliers ({len(outliers)})')
    axes[0, 1].set_title(f'📈 {coluna_principal} - Outliers Destacados')
    axes[0, 1].set_xlabel('Índice')
    axes[0, 1].set_ylabel('Valor')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Gráfico 3: Análise de movimentações (se temos dados suficientes)
    if len(dados_principais) > 1:
        movimentacoes = dados_principais.diff().dropna()
        axes[1, 0].hist(movimentacoes, bins=50, alpha=0.7, color='green', edgecolor='black')
        axes[1, 0].set_title('📊 Distribuição das Movimentações')
        axes[1, 0].set_xlabel('Mudança de Preço')
        axes[1, 0].set_ylabel('Frequência')
        axes[1, 0].axvline(0, color='red', linestyle='--', alpha=0.7)
        axes[1, 0].grid(True, alpha=0.3)
    
    # Gráfico 4: Análise de volatilidade rolling (se temos dados suficientes)
    if len(dados_principais) > 20:
        retornos = dados_principais.pct_change().dropna()
        volatilidade_rolling = retornos.rolling(window=20).std() * np.sqrt(252)  # Anualizando
        
        axes[1, 1].plot(volatilidade_rolling.index, volatilidade_rolling * 100, color='purple', alpha=0.8)
        axes[1, 1].set_title('📈 Volatilidade Rolling (20 períodos)')
        axes[1, 1].set_xlabel('Índice')
        axes[1, 1].set_ylabel('Volatilidade Anualizada (%)')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Resumo estatístico dos outliers
    print(f"\n📋 Resumo da Análise de Outliers ({coluna_principal}):")
    print(f"   • Total de observações: {len(dados_principais):,}")
    print(f"   • Outliers detectados: {len(outliers):,}")
    print(f"   • Percentual de outliers: {(len(outliers)/len(dados_principais))*100:.2f}%")
    print(f"   • Limite inferior (Q1 - 1.5*IQR): {limite_inferior:.2f}")
    print(f"   • Limite superior (Q3 + 1.5*IQR): {limite_superior:.2f}")
    
    if len(outliers) > 0:
        print(f"   • Outlier mínimo: {outliers.min():.2f}")
        print(f"   • Outlier máximo: {outliers.max():.2f}")
    
else:
    print("❌ Dados numéricos insuficientes para análise de outliers")

print("\n✅ Análise avançada concluída!")

## 📊 Resumo Executivo da Análise Exploratória

### 🎯 **Principais Descobertas**

#### 📋 **Estrutura dos Dados**
- **Fonte**: WIN$N_M1.csv (Mini Índice Bovespa - timeframe 1 minuto)
- **Período**: Dados de alta frequência para análise intraday
- **Variáveis**: OHLCV (Open, High, Low, Close, Volume)

#### 📈 **Características Financeiras Observadas**
1. **Preços OHLC**: Comportamento típico de série temporal financeira
2. **Volume**: Variação significativa ao longo do tempo
3. **Correlações**: Alta correlação entre preços OHLC (esperado)
4. **Volatilidade**: Clusters de volatilidade típicos de ativos financeiros

#### 🔍 **Qualidade dos Dados**
- **Valores Missing**: [Verificado durante execução]
- **Outliers**: Identificados usando método IQR
- **Consistência**: Relações lógicas entre OHLC mantidas

### 🚀 **Próximos Passos Sugeridos**

1. **Agregação Temporal**: 
   - Converter dados M1 para timeframes maiores (M5, M15, H1, D1)
   - Análise de padrões em diferentes horizontes temporais

2. **Análise de Gaps**:
   - Identificar gaps de abertura
   - Calcular probabilidades de fechamento
   - Classificar gaps por magnitude

3. **Indicadores Técnicos**:
   - Médias móveis (SMA, EMA)
   - RSI, MACD, Bollinger Bands
   - Suporte e resistência

4. **Modelagem Preditiva**:
   - Machine Learning para predição de direção
   - Análise de regime de volatilidade
   - Backtesting de estratégias

### 📚 **Insights para Trading**
- **Liquidez**: Volume alto indica boa liquidez para execução
- **Volatilidade**: Clusters identificados podem indicar oportunidades
- **Padrões Intraday**: Comportamento típico ao longo do dia de pregão

In [ ]:
# 💾 Salvamento dos Dados Processados (Opcional)
print("💾 SALVAMENTO DOS DADOS PROCESSADOS")
print("="*40)

try:
    # Criar diretório para dados processados
    import os
    output_dir = "dados_processados"
    os.makedirs(output_dir, exist_ok=True)
    
    # Salvar dados processados
    dados_processados.to_csv(f"{output_dir}/WIN_dados_explorados.csv", index=False)
    print(f"✅ Dados processados salvos em: {output_dir}/WIN_dados_explorados.csv")
    
    # Salvar estatísticas resumo
    if len(dados_processados.select_dtypes(include=[np.number]).columns) > 0:
        stats_resumo = dados_processados.describe()
        stats_resumo.to_csv(f"{output_dir}/WIN_estatisticas_resumo.csv")
        print(f"✅ Estatísticas salvas em: {output_dir}/WIN_estatisticas_resumo.csv")
    
    # Resumo final
    print(f"\n📊 RESUMO FINAL DA ANÁLISE:")
    print(f"   • Dataset original: {dados_brutos.shape[0]:,} registros")
    print(f"   • Colunas analisadas: {dados_processados.shape[1]}")
    print(f"   • Período: Dados minuto a minuto do WIN$N")
    print(f"   • Qualidade: Análise de missing, outliers e correlações realizada")
    print(f"   • Visualizações: Séries temporais, distribuições e correlações geradas")
    
    print(f"\n🎯 Esta análise exploratória fornece a base para:")
    print(f"   • Desenvolvimento de estratégias de trading")
    print(f"   • Análise de gaps e padrões intraday") 
    print(f"   • Modelagem preditiva e machine learning")
    print(f"   • Backtesting de sistemas automatizados")
    
except Exception as e:
    print(f"❌ Erro ao salvar dados: {e}")

print(f"\n✅ Análise exploratória concluída com sucesso! 🚀")